In [ ]:
#Python ver. 확인 - 3.7 이상 or 이하 코드 다름
import sys
print(sys.version)

In [ ]:
#설치 코드
!pip install bs4
!pip install selenium

In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
from tqdm import tqdm

from selenium.webdriver.common.by import By #python ver.3.7 이상일 경우 추가 필요

In [ ]:
#기간 설정
first_days = pd.date_range('2021/11/01', '2022/10/31', freq='MS')
last_days = pd.date_range('2021/11/01', '2022/10/31', freq='M')

#키워드 설정 - 단독 키워드 or 네이버 검색 연산자 사용 결합 키워드 입력
self_keyword = ["(대학생 | 과제 | 시험 | 팀플 | 종강 | 개강 | 밤샘 | 공강 | 과방 | 동방 | 랩실 | 중도 | 학식) +점메추"]  

In [ ]:
#크롤링 범위
#블로그 (카페제외)
#본문 (제목 & 댓글 제외)
#기간: 포스트 코로나 - 최근 1년 (2021. 11. 01 ~ 2022. 10. 31)
#네이버 View가 한 번에 1000개까지 검색되는 점을 고려하여, 날짜를 월 단위로 쪼개어 매월 당 최대 1000개씩을 크롤링함

#은지 할당 키워드 - "쇼핑"
#쇼핑 + (대학생 | 용돈 | 알바 | 장학금 | 새내기 | 복학생 | 자취생 | 수업 끝 | 오티 | 엠티)
#Self keyword = 지그재그, 무신사, 에이블리, 브랜디, 개강룩, 새내기룩, OOTD
#Sub keyword = 언박싱, 친환경, -로그, 손민수, 왓츠인마이백, 랜선집들이, 불매, 장비빨, 앱등이/삼엽충, 당근/쿨거래/네고/매너온도, 봄웜/여쿨/갈웜/겨쿨, 

### 요청한 만큼 스크롤이 내려간 VIEW 페이지에서 url을 긁어오기

In [ ]:
blog_url_list=[]

for keyword in self_keyword:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
    
    driver = webdriver.Chrome('./chromedriver.exe') #크롬 드라이버 위치 변경 필요
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 1020  #default:1020
    
    driver.find_element(By.XPATH,r'//*[@id="snb"]/div[1]/div/div[2]/a').click() #python ver.3.7 이상: driver.find_element(By.XPATH,
    
    for k in range(len(first_days)):
        try:
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element(By.XPATH,'//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
        
            last_height = driver.execute_script("return document.body.scrollHeight") 

            scroll = (url_want/30)-1 
        
            for i in range(int(scroll)):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(random.uniform(1,1.7))
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:   
                    break
                                                                             
                last_height = new_height
                                    
            soup = bs(driver.page_source, 'lxml') 
            blog_url = soup.find_all('a',class_='api_txt_lines total_tit')

            for i in blog_url:
                blog_url_list.append(i['href'])
            
            print('{0} 키워드 {1}년{2}월'.format(keyword, first_days.year[k], first_days.month[k]), len(blog_url),' 개 블로그 url주소 수집')

        
            driver.execute_script("window.scrollTo(0, 0)")
        
        except:
            pass

### 수집 후 blog_url_list를 set type으로 전환해 혹시 중복된 url이 있는지 확인

In [ ]:
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

### #본문 & url 수집

In [ ]:
no_naver_blog1=[]

start = time.time()

#본문 & url 수집
time_list = []
review_list = []
url_list = []
count = 0

driver = webdriver.Chrome('./chromedriver.exe')  #크롬 드라이버 위치 변경 필요
for url in tqdm(blog_url_list): #기본적으로 고치지 않지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고침
    if 'naver' in url:
    
        count += 1
        driver.get(url)
        time.sleep(1.1)

        try:
            
            driver.switch_to.frame('mainFrame') 
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') 
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]')
            postview_reg = reg.sub(' ',postview)

            try:
                timeline = driver.find_element(By.XPATH,"//span[@class='se_publishDate pcol2']")
        
            except:
                timeline = driver.find_element(By.XPATH,"//p[@class='date fil5 pcol2 _postAddDate']")
            timeline = timeline.text

    
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            url_list.append(url)
            time.sleep(random.uniform(1,1.7))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))
df

In [ ]:
data = {'time' : time_list,
        'review' : review_list,
        'url': url_list}
df = pd.DataFrame(data)

print('걸린 시간: ', time.time()-start)
print('최종 {}개 블로그 크롤링 완료'.format(count))
df

In [ ]:
#csv 파일로 저장
df.to_csv("점메추_성률1.csv", encoding = "utf-8-sig") #파일 이름 변경 필요 "title_keyword"